In [1]:
#Adapted from   /mnt/data/supratik/demonstrateDIR/v2_evalDIRParamDataGeneration.py
import argparse
from argparse import Namespace
import os
import re
from functools import reduce
from glob import glob
from time import sleep
import json
import csv
import nibabel as nib

import imageio #Bose: Imageio is a Python library that provides an easy interface to read and write a wide range of image data, including animated images, volumetric data
import matplotlib.pyplot as plt
import matplotlib.cm as cm #Bose: matplotlib colormaps and functions
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize #Bose: The matplotlib.colors module is used for converting color or numbers arguments to RGBA or RGB.This module is used for mapping numbers to colors or color specification conversion in a 1-D array of colors also known as colormap.And Normalize class is used to normalize data into the interval of [0.0, 1.0].
from skimage.segmentation import mark_boundaries #Bose: Return image with boundaries between labeled regions highlighted
from skimage.transform import rescale #Bose: Rescale operation resizes an image by a given scaling factor. The scaling factor can either be a single floating point value, or multiple values - one along each axis.
colormap = cm.hsv
norm = Normalize()

import pydicom
from scipy.ndimage import morphology

from torch.nn import MSELoss
from viu.io import volume
from viu.io.volume import read_volume
from viu.torch.deformation.fields import DVF, set_identity_mapping_cache
from viu.torch.io.deformation import *
from viu.util.body_mask import seg_body
from viu.util.memory import fmt_mem_size
from viu.util.config import json_config
from viu.torch.visualization.ortho_utils import save_ortho_views #from pamomo.visualization.ortho_utils import save_ortho_views
from viu.torch.measure.voi import measure_voi_list

from pamomo.pca.cmo_pca import CMoPCA
from pamomo.registration.deformable import reg, force_unload
from pamomo.visualization.cmo_pca_plots import *
from pamomo.metrices.residual_deformation import *

In [2]:
from exampleUtils import *
mse = MSELoss()

In [3]:
workingFolderParent = '/home/wd974888/Downloads'
patiendId = 1 #in range(8,9): #range(1,12) #range(1,12) #Use <truncateDepth_initial, final> : Pat01:<75, 50>;  Pat04:<0, 0>; Pat07:<95, 95>; Pat09:<200, 100>;
binningType = 'PB' # in ['AB']: #['AB', 'PB']
patientMRN = f'Patient{patiendId:02d}{binningType}'
print(f'patientMRN {patientMRN}')
truncateDepth_initial=75
truncateDepth_final=50
currentDirOptionsKey = f"config01" #in [f"config{i:02d}" for i in range(10,11)]: #[f"config{i:02d}" for i in range(1,11)]
print(f'currentDirOptionsKey {currentDirOptionsKey}')

patientMRN Patient01PB
currentDirOptionsKey config01


In [4]:
#Get all the paths

org_vols_hdf_path = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/{patientMRN}_test_vols.hdf'
print(f'org_vols_hdf_path {org_vols_hdf_path} exists:{os.path.exists(org_vols_hdf_path)}')

cache_fn =  f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Cache/Interpol_{currentDirOptionsKey}_{patientMRN}_test_pca.pt'
print(f'Interpol hardcoded cache file name: {cache_fn} exists:{os.path.exists(cache_fn)}')

functional_pca_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Functional_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
print(f'functional_pca_fn: {functional_pca_fn} exists:{os.path.exists(functional_pca_fn)}')

functional_reconstructedVol_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Functional_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
print(f'functional_reconstructedVol_fn: {functional_reconstructedVol_fn} exists:{os.path.exists(functional_reconstructedVol_fn)}')

intFuncMixed_pca_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/IntFuncMixed_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
print(f'intFuncMixed_pca_fn: {intFuncMixed_pca_fn} exists:{os.path.exists(intFuncMixed_pca_fn)}')

intFuncMixed_reconstructedVol_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/IntFuncMixed_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
print(f'intFuncMixed_reconstructedVol_fn: {intFuncMixed_reconstructedVol_fn} exists:{os.path.exists(intFuncMixed_reconstructedVol_fn)}')

interpol_pca_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Interpol_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
print(f'interpol_pca_fn: {interpol_pca_fn} exists:{os.path.exists(interpol_pca_fn)}')

interpol_reconstructedVol_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Interpol_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
print(f'interpol_reconstructedVol_fn: {interpol_reconstructedVol_fn} exists:{os.path.exists(interpol_reconstructedVol_fn)}')


org_vols_hdf_path /home/wd974888/Downloads/workingFolder/DeformationExperiment/PCA/Patient01PB/Patient01PB_test_vols.hdf exists:True
Interpol hardcoded cache file name: /home/wd974888/Downloads/workingFolder/DeformationExperiment/PCA/Patient01PB/Cache/Interpol_config01_Patient01PB_test_pca.pt exists:True
functional_pca_fn: /home/wd974888/Downloads/workingFolder/DeformationExperiment/PCA/Patient01PB/Functional_config01_Patient01PB_test_pca.hdf exists:True
functional_reconstructedVol_fn: /home/wd974888/Downloads/workingFolder/DeformationExperiment/PCA/Patient01PB/Functional_config01_Patient01PB_reconstructed_vols.hdf exists:False
intFuncMixed_pca_fn: /home/wd974888/Downloads/workingFolder/DeformationExperiment/PCA/Patient01PB/IntFuncMixed_config01_Patient01PB_test_pca.hdf exists:True
intFuncMixed_reconstructedVol_fn: /home/wd974888/Downloads/workingFolder/DeformationExperiment/PCA/Patient01PB/IntFuncMixed_config01_Patient01PB_reconstructed_vols.hdf exists:False
interpol_pca_fn: /home/wd9

In [5]:
vols_o, res_o, pos_o = volume.read_volume(org_vols_hdf_path , hdf_ds_name='volumes')
print(f'vols_o shape {vols_o.shape} dtype {vols_o.dtype} min {np.min(vols_o)} max {np.max(vols_o)}')
print(f'res_o {res_o}')
print(f'pos_o {pos_o}')

vols_o shape (10, 292, 512, 512) dtype float32 min -8192.0 max 33356.0
res_o [0.9765625 0.9765625 1.       ]
pos_o [ 1.718750e-03 -2.664983e+02  2.110000e+01]


In [8]:
vols, dvfs, res, pos, dvf_res, dvf_pos, msks, vol_idx_msk = torch.load(cache_fn)
print(f'vols shape {vols.shape} dtype {vols.dtype} device {vols.device}  min {torch.min(vols)} max {torch.max(vols)}') #
print(f'res {res}')
print(f'pos {pos}')
print(f'dvfs shape {dvfs.shape} dtype {dvfs.dtype} device {dvfs.device} min {torch.min(dvfs)} max {torch.max(dvfs)}') #
print(f'dvf_res {dvf_res}')
print(f'dvf_pos {dvf_pos}')

vols shape torch.Size([10, 292, 512, 512]) dtype torch.float32 device cpu  min -8192.0 max 33356.0
res [0.9765625 0.9765625 1.       ]
pos [ 1.71875000e-03 -2.66498281e+02  2.11000000e+01]
dvfs shape torch.Size([1, 10, 146, 256, 256, 3]) dtype torch.float32 device cpu min DVF(-0.0612) max DVF(0.0739)
dvf_res tensor([1.9570, 1.9570, 2.0069], dtype=torch.float64)
dvf_pos tensor([0., 0., 0.], dtype=torch.float64)


In [11]:
functional_reconstructed_vols, functional_reconstructed_res, functional_reconstructed_pos = volume.read_volume(functional_reconstructedVol_fn, hdf_ds_name='volumes')
print(f'functional_reconstructed_vols shape {functional_reconstructed_vols.shape} dtype {functional_reconstructed_vols.dtype}   min {np.min(functional_reconstructed_vols)} max {np.max(functional_reconstructed_vols)}') #
print(f'functional_reconstructed_res {functional_reconstructed_res}')
print(f'functional_reconstructed_pos {functional_reconstructed_pos}')
# reconstructed_vols=torch.tensor(reconstructed_vols, device=vols.device)

functional_reconstructed_vols shape (10, 292, 512, 512) dtype float32   min -4717.4619140625 max 32484.1875
functional_reconstructed_res [0.9765625 0.9765625 1.       ]
functional_reconstructed_pos [ 1.718750e-03 -2.664983e+02  2.110000e+01]


In [12]:
intFuncMixed_reconstructed_vols, intFuncMixed_reconstructed_res, intFuncMixed_reconstructed_pos = volume.read_volume(intFuncMixed_reconstructedVol_fn, hdf_ds_name='volumes')
print(f'intFuncMixed_reconstructed_vols shape {intFuncMixed_reconstructed_vols.shape} dtype {intFuncMixed_reconstructed_vols.dtype}   min {np.min(intFuncMixed_reconstructed_vols)} max {np.max(intFuncMixed_reconstructed_vols)}') #
print(f'intFuncMixed_reconstructed_res {intFuncMixed_reconstructed_res}')
print(f'intFuncMixed_reconstructed_pos {intFuncMixed_reconstructed_pos}')


intFuncMixed_reconstructed_vols shape (10, 292, 512, 512) dtype float32   min -5639.95263671875 max 34518.71875
intFuncMixed_reconstructed_res [0.9765625 0.9765625 1.       ]
intFuncMixed_reconstructed_pos [ 1.718750e-03 -2.664983e+02  2.110000e+01]


In [13]:
interpol_reconstructed_vols, interpol_reconstructed_res, interpol_reconstructed_pos = volume.read_volume(interpol_reconstructedVol_fn, hdf_ds_name='volumes')
print(f'interpol_reconstructed_vols shape {interpol_reconstructed_vols.shape} dtype {interpol_reconstructed_vols.dtype}   min {np.min(interpol_reconstructed_vols)} max {np.max(interpol_reconstructed_vols)}') #
print(f'interpol_reconstructed_res {interpol_reconstructed_res}')
print(f'interpol_reconstructed_pos {interpol_reconstructed_pos}')

interpol_reconstructed_vols shape (10, 292, 512, 512) dtype float32   min -4186.0654296875 max 31961.224609375
interpol_reconstructed_res [0.9765625 0.9765625 1.       ]
interpol_reconstructed_pos [ 1.718750e-03 -2.664983e+02  2.110000e+01]


In [ ]:
# Let us display one bin
from exampleUtils import *
display_vol = vols[0,...].cpu().numpy()
v1_volumeComparisonViewer3D(
    listVolumes=[display_vol],listLabels=[f'{patientMRN}_bin00'],
    maxZ0=display_vol.shape[0], maxZ1=display_vol.shape[1], maxZ2=display_vol.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500, useAspectCol=False)